## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [1]:
import pandas as pd

df = pd.read_csv('Resources/client_dataset.csv')

df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,item_id,category,subcategory,unit_price,unit_cost,unit_weight,qty,line_number
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,EUD29711-63-6U,decor,wall art,1096.80,762.71,7.50,105,1
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,XDA18116-89-4A,consumables,pens,24.95,15.09,1.49,21,0
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,ABE59463-05-7E,software,project management,13.52,7.86,1.68,39,6
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,ZMM00836-65-0C,consumables,pens,36.42,24.85,1.23,29,3
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,BZX55559-12-3X,consumables,misc,195.10,108.17,46.43,20,1


In [2]:
print("Column Names:")
print(df.columns)



Column Names:
Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')


In [37]:
print("Basic Statistics:")
print(df.describe())



Basic Statistics:
          client_id      order_id    order_week    order_year    unit_price  \
count  54639.000000  5.463900e+04  54639.000000  54639.000000  54639.000000   
mean   54837.869416  5.470190e+06     11.359139   2022.993064    136.267207   
std    25487.438231  2.599807e+06      7.023499      0.082997    183.873135   
min    10033.000000  1.000886e+06      1.000000   2022.000000      0.010000   
25%    33593.000000  3.196372e+06      6.000000   2023.000000     20.800000   
50%    53305.000000  5.496966e+06     11.000000   2023.000000     68.310000   
75%    78498.000000  7.733869e+06     17.000000   2023.000000    173.160000   
max    99984.000000  9.998480e+06     52.000000   2023.000000   1396.230000   

          unit_cost   unit_weight           qty   line_number      subtotal  \
count  54639.000000  54639.000000  5.463900e+04  54639.000000  5.463900e+04   
mean      99.446073      5.004116  5.702646e+02      2.979667  7.710286e+04   
std      133.164267      5.326599

In [41]:

unique_values = df.nunique()
print("Unique Values:")
print(unique_values)


Unique Values:
first                     360
last                      477
job                       498
phone                     999
email                     999
client_id                 999
order_id                10000
order_date                150
order_week                 23
order_year                  2
item_id                   500
category                    5
subcategory                18
unit_price                490
unit_cost                 487
unit_weight               393
qty                      2911
line_number                10
subtotal                41717
total_weight            30079
shipping_price          29758
shipping_cost           29758
total_price             42514
total_price_with_tax    42700
line_cost               41343
line_profit             40918
dtype: int64


In [42]:
top_categories = df['category'].value_counts().head(3)
print("Top three item categories with the most entries:")
print(top_categories)


Top three item categories with the most entries:
category
consumables    23538
furniture      11915
software        8400
Name: count, dtype: int64


In [2]:
most_entries_category = df['category'].value_counts().idxmax()

category_df = df[df['category'] == most_entries_category]

most_entries_subcategory = category_df['subcategory'].value_counts().head(1)

print("Subcategory with the most entries in the category with the most entries:")
print(most_entries_subcategory)


Subcategory with the most entries in the category with the most entries:
subcategory
bathroom supplies    6424
Name: count, dtype: int64


In [3]:
top_clients = df['client_id'].value_counts().head()
print("Top five clients with the most entries:")
print(top_clients)


Top five clients with the most entries:
client_id
33615    220
66037    211
46820    209
38378    207
24741    207
Name: count, dtype: int64


In [5]:
top_client_ids = top_clients.index.tolist()
print(top_client_ids)


[33615, 66037, 46820, 38378, 24741]


In [46]:
total_units_top_client = df[df['client_id'] == top_client_ids[0]]['qty'].sum()
print("Total units ordered by the client with the most entries:", total_units_top_client)


Total units ordered by the client with the most entries: 64313


## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [10]:
# Create a column that calculates the 
# subtotal for each line using the unit_price
# and the qty
df['subtotal'] = df['unit_price'] * df['qty']
print(df[['unit_price', 'qty', 'subtotal']].head(2))



   unit_price  qty   subtotal
0     1096.80  105  115164.00
1       24.95   21     523.95


In [19]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound
# for orders over 50 pounds and $10 per
# pound for items 50 pounds or under.

def calculate_shipping_cost(total_weight):
    if total_weight > 50:
        return total_weight * 7  
    else:
        return total_weight * 10  

df['shipping_price'] = df['total_weight'].apply(calculate_shipping_cost)

df['total_weight'] = df['unit_weight'] * df['qty']

print(df[['unit_price', 'unit_weight', 'qty', 'total_weight', 'shipping_price']].head())




   unit_price  unit_weight  qty  total_weight  shipping_price
0     1096.80         7.50  105        787.50         5512.50
1       24.95         1.49   21         31.29          312.90
2       13.52         1.68   39         65.52          458.64
3       36.42         1.23   29         35.67          356.70
4      195.10        46.43   20        928.60         6500.20


In [45]:
# Create a column for the total price
# using the subtotal and the shipping price
# along with a sales tax of 9.25%

sales_tax_rate = 0.0925

df['total_price'] = df['subtotal'] + df['shipping_price']
df['total_price_with_tax'] = df['total_price'] * (1 + sales_tax_rate)
df['line_price'] = df['line_subtotal'] + df['shipping_price']

print(df[['line_subtotal', 'shipping_price', 'line_price', 'total_price', 'total_price_with_tax']].head(3))



   line_subtotal  shipping_price  line_price  total_price  \
0      115164.00         5512.50   120676.50    120676.50   
1         523.95          312.90      836.85       836.85   
2         527.28          458.64      985.92       985.92   

   total_price_with_tax  
0         131839.076250  
1            914.258625  
2           1077.117600  


In [43]:
# Create a column for the cost
# of each line using unit cost, qty, and
# shipping price (assume the shipping cost
# is exactly what is charged to the client).

df['line_cost'] = df['unit_cost'] * df['qty']

df['total_price'] = df['line_cost'] + df['shipping_price']

print(df[['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id', 'order_date', 'order_week', 'order_year',
        'unit_price', 'unit_cost', 'unit_weight', 'qty', 'line_number', 'line_subtotal', 'total_weight',
        'shipping_price', 'line_price', 'line_cost']].head(3).to_string(index=False, float_format="{:.2f}".format))

  first    last             job              phone                             email  client_id  order_id order_date  order_week  order_year  unit_price  unit_cost  unit_weight  qty  line_number  line_subtotal  total_weight  shipping_price  line_price  line_cost
 Donald Harding    Immunologist 793-904-7725x39308  harding.donald.7185@sullivan.com      58515   8953482 2023-04-28          17        2023     1096.80     762.71         7.50  105            1      115164.00        787.50         5512.50   120676.50   80084.55
Tiffany   Myers Music therapist   201.442.4543x942 myers.t.6537@ferguson-johnson.net      37609   8069089 2023-05-19          20        2023       24.95      15.09         1.49   21            0         523.95         31.29          312.90      836.85     316.89
Shannon  Watson    Immunologist  687.737.9424x8503             swatson8146@payne.net      57113   1902144 2023-01-29           4        2023       13.52       7.86         1.68   39            6         527.28  

In [42]:
# Create a column for the profit of
# each line using line cost and line price
df['line_profit'] = df['subtotal'] - df['line_cost']

#print(df)
print(df[['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id', 'order_date', 'order_week', 'order_year',
        'unit_price', 'unit_cost', 'unit_weight', 'qty', 'line_number', 'line_subtotal', 'total_weight',
        'shipping_price', 'line_price', 'line_cost', 'line_profit']].head(3).to_string(index=False, float_format="{:.2f}".format))



  first    last             job              phone                             email  client_id  order_id order_date  order_week  order_year  unit_price  unit_cost  unit_weight  qty  line_number  line_subtotal  total_weight  shipping_price  line_price  line_cost  line_profit
 Donald Harding    Immunologist 793-904-7725x39308  harding.donald.7185@sullivan.com      58515   8953482 2023-04-28          17        2023     1096.80     762.71         7.50  105            1      115164.00        787.50         5512.50   120676.50   80084.55     35079.45
Tiffany   Myers Music therapist   201.442.4543x942 myers.t.6537@ferguson-johnson.net      37609   8069089 2023-05-19          20        2023       24.95      15.09         1.49   21            0         523.95         31.29          312.90      836.85     316.89       207.06
Shannon  Watson    Immunologist  687.737.9424x8503             swatson8146@payne.net      57113   1902144 2023-01-29           4        2023       13.52       7.86         

## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [49]:
# Check your work using the totals above
order_ids_to_check = [2742071, 2173913, 6128929]
filtered_orders = df[df['order_id'].isin(order_ids_to_check)].copy()

filtered_orders['total_price_with_tax'] = filtered_orders['total_price_with_tax'].fillna(0)
total_prices = filtered_orders.groupby('order_id')['total_price_with_tax'].sum()

print("Calculated Total Prices:")
for order_id, total_price in total_prices.items():
    print(f"Order {order_id} total: ${total_price:.2f}")



Calculated Total Prices:
Order 2173913 total: $162388.72
Order 2742071 total: $152811.90
Order 6128929 total: $923441.24


## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [58]:
# How much did each of the top 5 clients by quantity
# spend? Check your work from Part 1 for client ids.

top_clients = df['client_id'].value_counts().head()

top_clients_orders = df[df['client_id'].isin(top_clients.index)].copy()

top_clients_orders['total_spent_with_tax'] = top_clients_orders['total_price_with_tax'].fillna(0)
total_spent_by_client = top_clients_orders.groupby('client_id')['total_spent_with_tax'].sum()

for client_id, total_spent in total_spent_by_client.items():
    print(f"{client_id}: ${total_spent:.2f}")


24741: $82268891.98
33615: $8377308.57
38378: $12906550.88
46820: $9743794.32
66037: $10259514.80


In [68]:
# Create a summary DataFrame showing the totals for the
# for the top 5 clients with the following information:
# total units purchased, total shipping price,
# total revenue, and total profit. Sort by total profit.

grouped_by_client = df.groupby('client_id')

summary_df = pd.DataFrame({
    'qty': grouped_by_client['qty'].sum(),
    'shipping_price': grouped_by_client['shipping_price'].sum(),
    'line_price': grouped_by_client['total_price'].sum(),
    'line_cost': grouped_by_client['line_cost'].sum(),
    'line_profit': grouped_by_client.apply(lambda x: (x['subtotal'] - x['line_cost']).sum())
})

top_5_clients_summary = summary_df.nlargest(5, 'line_profit')

print("client_id\tqty\tshipping_price\tline_price\tline_cost\tline_profit")
for client_id, row in top_5_clients_summary.iterrows():
    print(f"{client_id}\t{row['qty']}\t{row['shipping_price']:.2f}\t{row['line_price']:.2f}\t{row['line_cost']:.2f}\t{row['line_profit']:.2f}")


client_id	qty	shipping_price	line_price	line_cost	line_profit
78965	4033302.0	5340756.88	600387798.68	403230070.71	191816971.09
85931	644423.0	10122549.19	217315333.11	118000323.41	89192460.51
24741	239862.0	5126448.37	75303333.62	40562451.34	29614433.91
38473	402770.0	4391003.73	168011507.38	135809841.41	27810662.24
86721	1010426.0	64335727.72	165918807.39	73955873.86	27627205.81


In [73]:
# Format the data and rename the columns
# to names suitable for presentation.
# Currency should be in millions of dollars.

# Convert the values to millions
top_5_clients_summary_millions = top_5_clients_summary / 1000000

# Create a new DataFrame with adjusted column names
adjusted_column_names = ['Total Units Purchased (Millions)', 
                        'Total Shipping Price (Millions)', 
                        'Total Revenue (Millions)', 
                        'Total Profit (Millions)']
formatted_summary = top_5_clients_summary_millions.rename(columns=dict(zip(top_5_clients_summary_millions.columns, adjusted_column_names)))

print("Formatted and Renamed Summary DataFrame for the top 5 clients:")
print(formatted_summary)


Formatted and Renamed Summary DataFrame for the top 5 clients:
           Total Units Purchased (Millions)  Total Shipping Price (Millions)  \
client_id                                                                      
78965                              4.033302                         5.340757   
85931                              0.644423                        10.122549   
24741                              0.239862                         5.126448   
38473                              0.402770                         4.391004   
86721                              1.010426                        64.335728   

           Total Revenue (Millions)  Total Profit (Millions)  line_profit  
client_id                                                                  
78965                    600.387799               403.230071   191.816971  
85931                    217.315333               118.000323    89.192461  
24741                     75.303334                40.562451    29.61443

In [36]:
# Sort the updated data by "Total Profit" form highest to lowest
top_5_clients_summary_sorted = top_5_clients_summary_millions.sort_values(by='Total Profit (Millions)', ascending=False)

print("Sorted Summary DataFrame by Total Profit (from highest to lowest):")
print(top_5_clients_summary_sorted)


Sorted Summary DataFrame by Total Profit (from highest to lowest):
           Total Units Purchased (Millions)  Total Shipping Price (Millions)  \
client_id                                                                      
78965                              4.033302                         5.340757   
85931                              0.644423                        10.122549   
24741                              0.239862                         5.126448   
38473                              0.402770                         4.391004   
86721                              1.010426                        64.335728   

           Total Revenue (Millions)  Total Profit (Millions)  
client_id                                                     
78965                    408.570828               191.816971  
85931                    128.122873                89.192461  
24741                     45.688900                29.614434  
38473                    140.200845                27.810